In [ ]:
import pandas as pd
from keybert import KeyBERT
import os
import ast

# KeyBERT 모델 로드
kw_model = KeyBERT(model='distiluse-base-multilingual-cased-v1')

# 파일 경로 설정
base_path = "processing_review"
file_paths = [
    os.path.join(base_path, "mosh_reviews_preprocessed_okt.csv"),
    os.path.join(base_path, "stanley_reviews_preprocessed_okt.csv"),
    os.path.join(base_path, "beanplus_reviews_preprocessed_okt.csv"),
    os.path.join(base_path, "kinto_reviews_preprocessed_okt.csv"),
    os.path.join(base_path, "konu_reviews_preprocessed_okt.csv"),
    os.path.join(base_path, "locknlock_reviews_preprocessed_okt.csv")
]

# 전체 브랜드 대표 키워드 저장용
brand_keyword_summary = {}

# 처리 시작
for file_path in file_paths:
    brand = os.path.basename(file_path).split("_")[0]
    print(f"[{brand}] 전체 리뷰 토큰 이어붙여 키워드 추출 중...")

    df = pd.read_csv(file_path)
    df["tokens_list"] = df["tokens"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # 전체 토큰 이어붙이기
    all_tokens = sum(df["tokens_list"], [])  # flatten
    full_text = " ".join(all_tokens)

    # KeyBERT로 키워드 추출 (브랜드 전체 기준)
    keywords = kw_model.extract_keywords(full_text, top_n=20)
    brand_keyword_summary[brand] = keywords

# 결과 출력
for brand, keywords in brand_keyword_summary.items():
    print(f"\n🔹 {brand.upper()} 대표 키워드:")
    for kw, score in keywords:
        print(f"  - {kw}: {round(score, 3)}")
